In [5]:
import os

path = "X:\Directory\code\Durable-Federated-Learning-Backdoor\FL_Backdoor_CV\cifar100_central_saved_model/Backdoor_saved_models_update1_noniid_100_cifar100_EC0_EE3801/maskratio0.95"
os.path.exists(path)
os.makedirs(path)

In [ ]:
import pickle
import numpy as np
from transformers import *
from image_helper import * 

with open('./data/southwest_images_new_train.pkl', 'rb') as train_f:
    saved_southwest_dataset_train = pickle.load(train_f)
    print(saved_southwest_dataset_train)
with open('./data/southwest_images_new_test.pkl', 'rb') as test_f:
    saved_southwest_dataset_test = pickle.load(test_f)

print('shape of edge case train data (southwest airplane dataset train)',saved_southwest_dataset_train.shape)
print('shape of edge case test data (southwest airplane dataset test)',saved_southwest_dataset_test.shape)

# np.ones((x,y), dype=int) 建立一个[x,y]维的int型数组，且值为1，再*9
sampled_targets_array_train = 9 * np.ones((saved_southwest_dataset_train.shape[0],), dtype =int)
print(sampled_targets_array_train)
sampled_targets_array_test = 9 * np.ones((saved_southwest_dataset_test.shape[0],), dtype =int)
print(np.max(saved_southwest_dataset_train))

transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

trainset = Customize_Dataset(X=saved_southwest_dataset_train, Y=sampled_targets_array_train, transform=transform)
poisoned_train_loader = DataLoader(dataset = trainset, batch_size = 64, shuffle = True, num_workers=1)
print(trainset)

testset = Customize_Dataset(X=saved_southwest_dataset_test, Y=sampled_targets_array_test, transform=transform)
poisoned_test_loader = DataLoader(dataset = testset, batch_size = 64, shuffle = True, num_workers=1)


In [ ]:
import torchvision
from torchvision import transforms


transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

transform_test = transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
                ])

trainset = torchvision.datasets.CIFAR10(root = './data',
                                        train = True,
                                        download = False,
                                        transform=None)

In [ ]:
from typing import Text
from yaml import tokens
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import MNIST, EMNIST

from helper import Helper
import random
from utils.text_load import Dictionary
from models.word_model import RNNModel
from models.resnet import ResNet18
from models.lenet import LeNet
from models.edge_case_cnn import Net
from models.resnet9 import ResNet9
from utils.text_load import *
import numpy as np
import copy
from torch.utils.data import Dataset, DataLoader

import os
from torchvision import datasets, transforms
from collections import defaultdict
from torch.utils.data import DataLoader, random_split, TensorDataset
import pickle
from image_helper import Customize_Dataset
from image_helper import *
from image_helper import get_poison_cifar10_train_label


transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
poison_cifar10_train = get_poison_cifar10()
test_dataset = datasets.CIFAR10('X:\Directory\code\Durable-Federated-Learning-Backdoor\FL_Backdoor_CV\data', train=False, transform=transform_test)
sampled_targets_array_test = get_poison_cifar10_train_label()


poison_testset = Customize_Dataset(X=poison_cifar10_train, Y=sampled_targets_array_test, transform=transform_test)
cifar_poison_classes_ind = []
label_list = []
print(poison_testset[0])
for ind, x in enumerate(poison_testset):
    imge, label = x
    label_list.append(label)
    if label == 5:
        cifar_poison_classes_ind.append(ind)


print(cifar_poison_classes_ind)

提取并打白底十字补丁触发器

In [ ]:
import os
import numpy as np
import pickle
# import imageio
import cv2 as cv 
import matplotlib.pyplot as plt
from PIL import Image as img
import argparse
import json


# 1 is dorm

start = 1
file = 'D:\code\code_xwd\dataset\cifar-10-batches-py\\data_batch_1'


# 解压缩，返回解压后的字典
def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict

label_dict = {
    0:'plane',
    1:'car',
    2:'bird',
    3:'cat',
    4:'deer',
    5:'dog',
    6:'frog',
    7:'horse',
    8:'ship',
    9:'truck'
}

# 显示测试集图片
dict = unpickle(file)
data = dict.get("data")
label = dict.get("labels")

poi_index = open('index_test.txt', 'a+')

for i in range(10, 11):
    image_m = np.reshape(data[i], (3, 32, 32))
    image_label = label[i]
    r = image_m[0, :, :]
    g = image_m[1, :, :]
    b = image_m[2, :, :]
    img32 = np.array(cv.merge([r, g, b]))

    """
    # 左上白块 4x4
    r[:5, :5] = 255
    g[:5, :5] = 255
    b[:5, :5] = 255
    # 白块中间十字
    r[2, 0:5] = 0
    r[0:5, 2] = 0
    g[2, 0:5] = 0
    g[0:5, 2] = 0
    b[2, 0:5] = 0
    b[0:5, 2] = 0
    """
    
    """
    # 右下白块 4x4
    r[27:, 27:] = 255
    g[27:, 27:] = 255
    b[27:, 27:] = 255
    """

    """
    # 左下白块 4x4
    r[27:, :5] = 255
    g[27:, :5] = 255
    b[27:, :5] = 255
    """

    """
    # 右上白块 4x4
    r[:5, 27:] = 255
    g[:5, 27:] = 255
    b[:5, 27:] = 255
    """

    img32_patch = np.array(cv.merge([r, g, b]))
    print(f"已打补丁：{i}")
    
    poi_index.write(str(i) + '  ' + label_dict[image_label] + '\n')
    

    plt.ion()
    plt.figure()
    plt.imshow(img32)   # cifar10 原图
    plt.axis('off')
    plt.xticks([])    # 去 x 轴刻度
    plt.yticks([])    # 去 y 轴刻度
    plt.savefig(f'D:/code/code_xwd/dataset/cifar-10-batches-py/pic/{i}.jpg',bbox_inches='tight', pad_inches = 0)
    plt.show()


In [ ]:
from typing import Text
from yaml import tokens
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import MNIST, EMNIST

from helper import Helper
import random
from utils.text_load import Dictionary
from models.word_model import RNNModel
from models.resnet import ResNet18
from models.lenet import LeNet
from models.edge_case_cnn import Net
from models.resnet9 import ResNet9
from utils.text_load import *
import numpy as np
import copy
from torch.utils.data import Dataset, DataLoader

import os
from torchvision import datasets, transforms
from collections import defaultdict
from torch.utils.data import DataLoader, random_split, TensorDataset
import pickle
from gradcam import GradCAM, GradCAMpp
from gradcam.utils import visualize_cam

class Customize_Dataset(Dataset):
    def __init__(self, X, Y, transform):
        self.train_data = X
        self.targets = Y
        self.transform = transform


    def __getitem__(self, index):
        data = self.train_data[index]
        target = self.targets[index]
        data = self.transform(data)

        return data, target


    def __len__(self):
        return len(self.train_data)


def get_poison_cifar10():
    with open('D:\code\code_xwd\dataset\patched-cifar-10\data_batch_1', 'rb') as train_1:
        poison_data1 = pickle.load(train_1)
    with open('D:\code\code_xwd\dataset\patched-cifar-10\data_batch_2', 'rb') as train_2:
        poison_data2 = pickle.load(train_2)
    with open('D:\code\code_xwd\dataset\patched-cifar-10\data_batch_3', 'rb') as train_3:
        poison_data3 = pickle.load(train_3)
    with open('D:\code\code_xwd\dataset\patched-cifar-10\data_batch_4', 'rb') as train_4:
        poison_data4 = pickle.load(train_4)
    with open('D:\code\code_xwd\dataset\patched-cifar-10\data_batch_5', 'rb') as train_5:
        poison_data5 = pickle.load(train_5)

    x1 = poison_data1.get('data').reshape(10000, 32, 32, 3)
    x2 = poison_data2.get('data').reshape(10000, 32, 32, 3)
    x3 = poison_data3.get('data').reshape(10000, 32, 32, 3)
    x4 = poison_data4.get('data').reshape(10000, 32, 32, 3)
    x5 = poison_data5.get('data').reshape(10000, 32, 32, 3)
    # x1 = np.row_stack((x1, x2))
    # x1 = np.row_stack((x1, x3))
    # x1 = np.row_stack((x1, x4))
    # x1 = np.row_stack((x1, x5))

    poison_cifar_train_data = x1
    
    return poison_cifar_train_data

def get_poison_cifar10_train_label():    
    with open('D:\code\code_xwd\dataset\patched-cifar-10\data_batch_1', 'rb') as train_1:
        poison_data1 = pickle.load(train_1)
    with open('D:\code\code_xwd\dataset\patched-cifar-10\data_batch_2', 'rb') as train_2:
        poison_data2 = pickle.load(train_2)
    with open('D:\code\code_xwd\dataset\patched-cifar-10\data_batch_3', 'rb') as train_3:
        poison_data3 = pickle.load(train_3)
    with open('D:\code\code_xwd\dataset\patched-cifar-10\data_batch_4', 'rb') as train_4:
        poison_data4 = pickle.load(train_4)
    with open('D:\code\code_xwd\dataset\patched-cifar-10\data_batch_5', 'rb') as train_5:
        poison_data5 = pickle.load(train_5)

    x1 = poison_data1.get('labels')
    x2 = poison_data2.get('labels')
    x3 = poison_data3.get('labels')
    x4 = poison_data4.get('labels')
    x5 = poison_data5.get('labels')
    # poison_cifar10_train_label = x1 + x2 + x3 + x4 + x5
    poison_cifar10_train_label = x1

    return poison_cifar10_train_label

transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
transform_org = transforms.Compose([
            transforms.ToTensor(),
        ])

poison_cifar10_train = get_poison_cifar10()
sampled_targets_poison_cifar10_train = get_poison_cifar10_train_label()
            
poison_trainset = Customize_Dataset(X=poison_cifar10_train, Y=sampled_targets_poison_cifar10_train, transform=transform_test)
poison_org = Customize_Dataset(X=poison_cifar10_train, Y=sampled_targets_poison_cifar10_train, transform=transform_org)            
poisoned_train_data = torch.utils.data.DataLoader(poison_trainset,
                               batch_size=1,
                               sampler=torch.utils.data.sampler.SubsetRandomSampler(
                                  [1]
                               ),)
poisoned_org_data = torch.utils.data.DataLoader(poison_org,
                               batch_size=1,
                               sampler=torch.utils.data.sampler.SubsetRandomSampler(
                                  [1]
                               ),)

image = torch.from_numpy(poison_cifar10_train[0])

# file = 'D:\code\code_xwd\dataset\patched-cifar-10\\data_batch_5'

# 解压缩，返回解压后的字典
def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict


dict = unpickle(file)


model = ResNet18(10)
model.cuda()

params = torch.load("D:\code\code_xwd\Durable-Federated-Learning-Backdoor\SAVE_MODEL\cifar10 patched attacknum 450\Backdoor_saved_models_update1_noniid_EC0_cifar10_Baseline_EE3801\Attacker_model_epoch_2180.pth")
model.load_state_dict(params)
criterion = torch.nn.CrossEntropyLoss()
data_iterator = poisoned_train_data
data_iterator_org = poisoned_org_data
optimizer = torch.optim.SGD(model.parameters(), lr=0.2,
                                                momentum=0.09,
                                                weight_decay=0.4)

grad_block = []	# 存放grad图
feaure_block = []	# 存放特征图




for batch_id, batch in enumerate(data_iterator):
    for batch_id_org, batch_org in enumerate(data_iterator_org):
        data_org, targets_org = batch_org
        data_org, targets_org = data_org.cuda(), targets_org.cuda()
    data, targets = batch
    data, targets = data.cuda(), targets.cuda()
    print(data.shape)
    output = model(data)
    gradcam = GradCAM.from_config(model_type='resnet', arch=model, layer_name='layer4')
    for i in range(0,10):
        mask, logit = gradcam(data, class_idx=i)
        heatmap, cam_result = visualize_cam(mask, data_org)

        plt.figure()
        plt.subplot(1,2,1)
        plt.imshow(transforms.ToPILImage()(heatmap))
        plt.subplot(1,2,2)
        plt.imshow(transforms.ToPILImage()(cam_result))

    plt.show()
    """
    loss = criterion(output, targets)
    model.requires_grad = True
    loss.backward(retain_graph=True)
    print(model.features)
    """
# print(torch.nn.Sequential(*list(model.children())[:-1]))


热力图

In [ ]:
from typing import Text
from yaml import tokens
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import MNIST, EMNIST

from helper import Helper
import random
from utils.text_load import Dictionary
from models.word_model import RNNModel
from models.resnet import ResNet18
from models.lenet import LeNet
from models.edge_case_cnn import Net
from models.resnet9 import ResNet9
from utils.text_load import *
import numpy as np
import copy
from torch.utils.data import Dataset, DataLoader

import os
from torchvision import datasets, transforms
from collections import defaultdict
from torch.utils.data import DataLoader, random_split, TensorDataset
import pickle
from gradcam import GradCAM, GradCAMpp
from gradcam.utils import visualize_cam
import PIL
import matplotlib.pyplot as plt

label_dict = {
    0:'plane',
    1:'car',
    2:'bird',
    3:'cat',
    4:'deer',
    5:'dog',
    6:'frog',
    7:'horse',
    8:'ship',
    9:'truck'
}
 
def main():
    model = ResNet18(10)
    model.cuda()

    params = torch.load("D:\code\code_xwd\Durable-Federated-Learning-Backdoor\SAVE_MODEL\cifar10 patched 90%敌手\Backdoor_saved_models_update1_noniid_EC0_cifar10_Baseline_EE3801\\target_model_epoch_1870.pth")
    model.load_state_dict(params)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.2,
                                                    momentum=0.09,
                                                    weight_decay=0.4)
    
    gradcam = GradCAM.from_config(model_type='resnet', arch=model, layer_name='layer4')
    for k in range (0, 100):
        
        # get an image and normalize with mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
        pil_img = PIL.Image.open(f'D:\code\code_xwd\dataset\patched-cifar-10\pic\\test\{k}.jpg')
        torch_img = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])(pil_img).cuda()
        normed_img = transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])(torch_img)[None]
        # get a GradCAM saliency map on the class index 10.
        output = model(normed_img)
        pred = output.data.max(1)[1]
        print(pred)
        
        
        
        plt.figure(dpi=50,figsize=(32,32))
        for i in range(0, 10):
            mask, logit = gradcam(normed_img, class_idx=i)
    
            # make heatmap from mask and synthesize saliency map using heatmap and img
            heatmap, cam_result = visualize_cam(mask, torch_img)
        
            
            # plt.subplot(1,2,1)
            
            # plt.imshow(transforms.ToPILImage()(heatmap))
            plt.subplot(1, 10, i+1)
            plt.axis("off")
            plt.imshow(transforms.ToPILImage()(cam_result))
            # plt.title(label_dict[i])
        plt.show()
        


if __name__ == '__main__':
    main()


置信度

In [50]:
from typing import Text
from yaml import tokens
import torch
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import MNIST, EMNIST

from helper import Helper
import random
from utils.text_load import Dictionary
from models.word_model import RNNModel
from models.resnet import ResNet18
from models.lenet import LeNet
from models.edge_case_cnn import Net
from models.resnet9 import ResNet9
from utils.text_load import *
import numpy as np
import copy
from torch.utils.data import Dataset, DataLoader

import os
from torchvision import datasets, transforms
from collections import defaultdict
from torch.utils.data import DataLoader, random_split, TensorDataset
import pickle
from gradcam import GradCAM, GradCAMpp
from gradcam.utils import visualize_cam
import PIL
import matplotlib.pyplot as plt
import cv2 as cv

label_dict = {
    0:'plane',
    1:'car',
    2:'bird',
    3:'cat',
    4:'deer',
    5:'dog',
    6:'frog',
    7:'horse',
    8:'ship',
    9:'truck'
}

def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='latin1')
    fo.close()
    return dict



def main():
    
    file = 'D:\code\code_xwd\Durable-Federated-Learning-Backdoor\data\cifar-10-batches-py\\test_batch'
    dict = unpickle(file)
    
    model = ResNet18(10)
    model.cuda()

    params = torch.load("D:\code\code_xwd\Durable-Federated-Learning-Backdoor\SAVE_MODEL\cifar10 poisoned attacknum 450\Backdoor_saved_models_update1_noniid_EC0_cifar10_Neurotoxin_GradMaskRation0.95_EE3801\Benign_user_976_model_epoch_2200.pth")
    model.load_state_dict(params)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.2,
                                                    momentum=0.09,
                                                    weight_decay=0.4)
    
    fo = open(file, 'rb')
    label = pickle.load(fo, encoding='latin1')
    fo.close()
    labels = label.get("labels")
    
    gradcam = GradCAM.from_config(model_type='resnet', arch=model, layer_name='layer4')
    
    sum = 0
    num_9 = 0
    for k in range (0, 100):
        
        # get an image and normalize with mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)
        # pil_img = PIL.Image.open(f'D:\code\code_xwd\dataset\cifar-10-batches-py\pic\\test\{k}.jpg')
        
        image_m = np.reshape(dict.get("data")[k], (3, 32, 32))
        r = image_m[0, :, :]
        g = image_m[1, :, :]
        b = image_m[2, :, :]
        img32 = np.array(cv.merge([r, g, b]))
        img32 = PIL.Image.fromarray(img32)
        torch_img = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])(img32).cuda()
        normed_img = transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010])(torch_img)[None]
        # get a GradCAM saliency map on the class index 10.
        output = model(normed_img)
        # print(output.data)
        pred = output.data.max(1)[1].tolist()
        probs = torch.nn.functional.softmax(output.cpu(), dim=1).detach().numpy()[0]
        print(probs)
        print(f"预测标签为：{pred}")
        print(f"真实标签为：[{labels[k]}]")

        if pred[0] == labels[k] and pred[0] != 9:
            print("预测与真实标签一致")
            sum = sum + 1
        if pred[0] == 9:
            num_9 += 1
        """
        plt.figure(dpi=50,figsize=(32,32))
        for i in range(0, 10):
            mask, logit = gradcam(normed_img, class_idx=i)
    
            # make heatmap from mask and synthesize saliency map using heatmap and img
            heatmap, cam_result = visualize_cam(mask, torch_img)
        
            
            # plt.subplot(1,2,1)
            
            # plt.imshow(transforms.ToPILImage()(heatmap))
            plt.subplot(1, 10, i+1)
            plt.axis("off")
            plt.imshow(transforms.ToPILImage()(cam_result))
            # plt.title(label_dict[i])
        plt.show()
        """
        
    print(f"预测正确的数量：{sum}")
    print(f"中毒的数量：{num_9}")
        
        
# train训练良性，中毒的用poison_test训练，

if __name__ == '__main__':
    main()


[0.03425863 0.04318353 0.06245413 0.1699009  0.03255006 0.0845219
 0.13470285 0.07976729 0.0419108  0.3167499 ]
预测标签为：[9]
真实标签为：[3]
[0.06730754 0.20490809 0.02398947 0.02117149 0.00942493 0.00997698
 0.02386998 0.03100929 0.10907407 0.4992681 ]
预测标签为：[9]
真实标签为：[8]
[0.0696612  0.15277053 0.05997246 0.06304356 0.02617542 0.02628813
 0.06588769 0.06740409 0.09323801 0.37555888]
预测标签为：[9]
真实标签为：[8]
[0.10082284 0.05414606 0.07889204 0.06945555 0.03005632 0.03129475
 0.08129629 0.06629924 0.07432238 0.4134145 ]
预测标签为：[9]
真实标签为：[0]
[0.04684453 0.05022673 0.10384899 0.15924348 0.04237236 0.07411666
 0.23582591 0.0688324  0.04364687 0.17504205]
预测标签为：[6]
真实标签为：[6]
预测与真实标签一致
[0.02324165 0.07485944 0.03428259 0.06838617 0.02449883 0.04586117
 0.07611578 0.10669661 0.02573309 0.5203247 ]
预测标签为：[9]
真实标签为：[6]
[0.02919453 0.14606908 0.04314349 0.05593135 0.02137807 0.03064087
 0.05913622 0.08214214 0.03052763 0.5018366 ]
预测标签为：[9]
真实标签为：[1]
[0.04082867 0.02104001 0.09126816 0.14288133 0.06132488 0.07